<a href="https://colab.research.google.com/github/OSGeoLabBp/tutorials/blob/master/hungarian/python/vectorization_hu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Vektorizáció

A vektorizációt a PYthon programok felgyorsítására használjuk. Használata alapvető nagy adathlmazok kezelése esetén.

A vektorizált Python kódban nincsenek ciklusok (melyek lassan futnak), helyette összetett adatszerkezeteken végzett múveleteket használunk mint például *numpy* tömbök, *pandas* adatsorok. Ezeket a modulokat (*numpy*. *pandas*, stb.) C/C++ nyelven írták, melyek sokkal hatékonyabban hajtják végre a szükséges ciklusokat.

A vektorizált kód nem csak gyorsabban fut, de a kód rövidebb, könnyebben olvasható és karbantartható lesz.

Nézzünk meg néhány példát vektorizálás nélküli és vektorizált megoldással.

Nagy vektorokat és mátrixokat használunk, hogy a két megoldás közötti különbség látványosabb legyen. A műveletek eredményét általában nem írjuk ki, hanem csak a futási időket.



## Vektor és skalár szorzet


Tíz millió elemből álló vektort szorzunk meg egy skalárral.

In [5]:
import numpy as np
import random
import time

# adatok inicializálása
n = 10_000_000              # vektor mérete
scalar = 2.564              # Skalár szorzó a vektorhoz

In [6]:
%%time
vlist = [random.random() for i in range(n)]  # véletszámokat tartalmazó vektor generálása (nem vektorizált)

CPU times: user 1.84 s, sys: 293 ms, total: 2.13 s
Wall time: 2.2 s


In [7]:
%%time
vect = np.random.rand(n)     # véletszámokat tartalmazó vektor generálása (vektorizált)

CPU times: user 81.5 ms, sys: 23.1 ms, total: 105 ms
Wall time: 106 ms


Nem vektorizált megoldás egyszerű ciklussal.

In [8]:
%%time
slist = []
for i in range(n):
    slist.append(vlist[i] * scalar)

CPU times: user 2.29 s, sys: 279 ms, total: 2.57 s
Wall time: 2.6 s


Nem vektorizált megoldás lista feldolgozással.

In [9]:
%%time
s1list = [v * scalar for v in vlist]

CPU times: user 604 ms, sys: 192 ms, total: 796 ms
Wall time: 798 ms


Vektorizált megoldás numpy tömbbel.

In [10]:
%%time
svect = vect * scalar

CPU times: user 12.1 ms, sys: 16 ms, total: 28.2 ms
Wall time: 29.3 ms


## Legnagyobb elem megkeresése a vektorban

Nem vektorizált megoldás Python ciklussal.

In [11]:
%%time
vmax = vlist[0]
for v in vlist[1:]:
    if v > vmax: vmax = v

CPU times: user 906 ms, sys: 92.6 ms, total: 998 ms
Wall time: 1.01 s


Vektorizált megoldás listával.

In [12]:
%%time
vmax = max(vlist)

CPU times: user 143 ms, sys: 1.72 ms, total: 145 ms
Wall time: 144 ms


Vektorizált megoldás numpy tömbbel.

In [13]:
%%time
vmax = np.max(vect)

CPU times: user 12.3 ms, sys: 0 ns, total: 12.3 ms
Wall time: 15 ms


## Legnagyobb abszolút értékű különbség megkeresése egy vektorban

Nem vektorizált megoldás.

In [14]:
%%time
max_dif = abs(vlist[0] - vlist[1])
for i in range(1, n):
    dif = abs(vlist[i-1] - vlist[i])
    if dif > max_dif: max_dif = dif

CPU times: user 3.2 s, sys: 12.4 ms, total: 3.21 s
Wall time: 3.23 s


Vektorizált megoldás numpy tömbbel.

In [15]:
%%time
max_dif = np.max(np.abs(vect[:-1] - vect[1:]))

CPU times: user 32.9 ms, sys: 103 ms, total: 136 ms
Wall time: 138 ms


## Mátrix soronkénti átlagának számítása

Adatok előkészítése, a korábbi vektorból mátrix készítése.

In [16]:
matrix = vect.reshape((5000, -1)) # 5000 soros mátrix
list_matrix = list(matrix)        # átalakítás listává a nem vektorizált feldolgozáshoz

In [17]:
%%time
row_means = []
for row in list_matrix:
    row_means.append(sum(row)/ len(row))

CPU times: user 997 ms, sys: 4.29 ms, total: 1 s
Wall time: 1.01 s


In [18]:
%%time
row_means = np.mean(matrix, axis=1)

CPU times: user 11.3 ms, sys: 0 ns, total: 11.3 ms
Wall time: 17.2 ms


## Területszámítás koordinátákból

Az alkalmazott képlet

$ 2 \cdot Area = \sum (x_i - x_{i+1}) \cdot (y_i + y_{i+1})$


Először létrehozunk egy sok töréspontból álló sokszöget.

In [20]:
from math import sin, cos, pi

Nem vektorizált megoldás az adatok generálására

In [26]:
%%time
# pontok generálása egy kör kerületén
c = []
r = 10
for i in range(0, 36000):   # Középponti szögek századfokokban
    x = r * sin(i / 18000 * pi)
    y = r * cos(i / 18000 * pi)
    c.append((x, y))

CPU times: user 25.9 ms, sys: 4.84 ms, total: 30.7 ms
Wall time: 33.2 ms


In [27]:
%%time
#non-vectorised solution
s = 0
n = coords.shape[0]
for i in range(n):
    j = i+1 if i < n-1 else 0
    s += (coords[i, 0] - coords[j, 0]) * (coords[i, 1] + coords[j, 1])
area1 = abs(s / 2)
print(f'Terület: {area1:.5f}')

Terület: 314.15926
CPU times: user 51.3 ms, sys: 0 ns, total: 51.3 ms
Wall time: 53.9 ms


Vektorizált megoldás az adatok generálására.

In [28]:
%%time
angle = np.arange(0, 360, 0.01) / 180 * pi  # szög radiánkban
crds = np.hstack((np.sin(angle).reshape(-1, 1) * r, np.cos(angle).reshape(-1, 1) * r))
crds1 = np.vstack((crds, crds[0]))    # add first point to the end

CPU times: user 3.29 ms, sys: 23 µs, total: 3.32 ms
Wall time: 3.33 ms


In [29]:
%%time
# vectorized solution, one liner
area2 = abs(np.sum((crds1[:-1, 0] - crds1[1:, 0]) * (crds1[:-1, 1] + crds1[1:, 1]))) / 2
print(f'Area vectorized {area2:.5f}')

Area vectorized 314.15926
CPU times: user 1.12 ms, sys: 0 ns, total: 1.12 ms
Wall time: 914 µs


Hasonlítsa össze a vektorizált és nem vektorizált megoldások futási idejét!

##Feladatok

1. Készítsen vektorizált megoldás egy vektor második differenciáinak előállítására!
2. Készítsen vektorizált megoldást egy négyzetes mátrix főátlójában lévő legnagyobb elem kikeresésére!
3. Készítsen vektorizált megoldást két vektor diadikus szorzatának előállítására!